In [2]:
import numpy as np
import pandas as pd

In [3]:
from dataclasses import dataclass
from typing import Optional


@dataclass
class Order:  # Our own placed order
    order_id: int
    side: str
    size: float
    price: float


@dataclass
class AnonTrade:  # Market trade
    timestamp: float
    side: str
    size: float
    price: float


@dataclass
class OwnTrade:  # Execution of own placed order
    timestamp: float
    trade_id: int
    order_id: int
    side: str
    size: float
    price: float


@dataclass
class OrderbookSnapshotUpdate:  # Orderbook tick snapshot
    timestamp: float
    asks: list[tuple]  # tuple[price, size]
    bids: list[tuple]


@dataclass
class MdUpdate:  # Data of a tick
    orderbook: Optional[OrderbookSnapshotUpdate] = None
    trades: Optional[list[AnonTrade]] = None

In [4]:
btc_trades = pd.read_csv("../../../Downloads/md/md/btcusdt_Binance_LinearPerpetual/trades.csv")
btc_lobs = pd.read_csv("../../../Downloads/md/md/btcusdt_Binance_LinearPerpetual/lobs.csv")

In [5]:
eth_trades = pd.read_csv("../../../Downloads/md/md/ethusdt_Binance_LinearPerpetual/trades.csv")
eth_lobs = pd.read_csv("../../../Downloads/md/md/ethusdt_Binance_LinearPerpetual/lobs.csv")

In [19]:
print(btc_trades.shape)
print(btc_lobs.shape)
print(eth_trades.shape)
print(eth_lobs.shape)

print(btc_trades.head())
print(np_btc_trades[0])

(5727714, 5)
(2541356, 42)
(4873803, 5)
(2539699, 42)
            receive_ts          exchange_ts aggro_side    price   size
0  1655942402624789714  1655942402623000000        BID  19977.5  0.001
1  1655942405293556247  1655942405292000000        BID  19977.5  0.041
2  1655942405293628020  1655942405292000000        BID  19977.5  0.036
3  1655942405293832021  1655942405292000000        BID  19977.5  0.001
4  1655942405293929517  1655942405292000000        BID  19977.5  0.001


In [26]:
import time

# btc_trades = pd.read_csv("../../../Downloads/md/md/btcusdt_Binance_LinearPerpetual/trades.csv")
# np_btc_trades = btc_trades.to_numpy()

sec1 = time.time()

sz = 0
for i in range(np_btc_trades.shape[0]):
    sz += btc_trades.iloc[i]['size']
    if i > 100 * 1000:
        break

sec2 = time.time()
print('Seconds %.4f, total size %.2f' % (sec2 - sec1, sz))
cur_sec = time.time()

sz = 0
for i in range(np_btc_trades.shape[0]):
    sz += np_btc_trades[i][4]
    if i > 100 * 1000:
        break

sec3 = time.time()
print('Seconds %.4f, total size %.2f' % (sec3 - sec2, sz))

Seconds 9.5663, total size 13141.92
Seconds 0.0399, total size 13141.92


In [14]:
cur_sec = time.time() * 1000 * 1000
for index in range(btc_trades.shape[0]):
    row = btc_trades.iloc[index]
    if index % 50000 == 0:
        print(index)
sec2 = time.time() * 1000 * 1000
print(sec2 - cur_sec)

0
50000
100000
150000
200000
250000
300000


KeyboardInterrupt: 

In [9]:
np_btc_trades = btc_trades.to_numpy()
np_btc_lobs = btc_lobs.to_numpy()

trade_columns = btc_trades.columns.to_numpy()
lobs_columns = btc_lobs.columns.to_numpy()

btc_ti = {trade_columns[i]: i for i, col in enumerate(trade_columns)}
btc_li = {lobs_columns[i]: i for i, col in enumerate(lobs_columns)}

In [17]:
cur_sec = time.time()
for i in range(btc_trades.shape[0]):
    d = np_btc_trades[i]
    if i % 50000 == 0:
        print(i)
sec2 = time.time()
print(sec2 - cur_sec)

313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
313579
2.320974588394165


In [10]:
def load_md_from_file(pref, trades, lobs, ti, li) -> list[MdUpdate]:
    # TODO: load actual md
    pref_ask_price = pref + 'ask_price_'
    pref_ask_size = pref + 'ask_vol_'
    pref_bid_price = pref + 'bid_price_'
    pref_bid_size = pref + 'bid_vol_'
    
    mds = []
    i_tr = 0
    
    print('Started parsing csv files...')
    for i_lobs in range(lobs.shape[0]):
        d = lobs[i_lobs]
        asks = [(d[li[pref_ask_price + str(i)]], d[li[pref_ask_size + str(i)]]) for i in range(10)]
        bids = [(d[li[pref_bid_price + str(i)]], d[li[pref_bid_size + str(i)]]) for i in range(10)]
        book = OrderbookSnapshotUpdate(timestamp = d[li[' exchange_ts']], asks = asks, bids = bids)
        
        anon_trades = []        
        while i_tr < trades.shape[0] and trades[i_tr][ti['exchange_ts']] < lobs[i_lobs][li[' exchange_ts']]:
            d = trades[i_tr]
            at = AnonTrade(timestamp=d[ti['exchange_ts']], side=d[ti['aggro_side']], size=d[ti['size']], price=d[ti['price']])    
            anon_trades.append(at)

            i_tr += 1
            if i_tr % 10000 == 0:
                print('Trade progress: %d, %.2f%%' % (i_tr, i_tr / trades.shape[0] * 100))
        
        mds.append(MdUpdate(orderbook = book, trades = anon_trades))
        
        if i_lobs % 10000 == 0:
            print('Lobs progress: %d, %.2f%%' % (i_lobs, i_lobs / lobs.shape[0] * 100))
            
    print('Finished!')
    return mds
                 

loaded_md = load_md_from_file('btcusdt:Binance:LinearPerpetual_', np_btc_trades, np_btc_lobs, btc_ti, btc_li)

Started parsing csv files...
Lobs progress: 0, 0.00%
Trade progress: 10000, 0.17%
Trade progress: 20000, 0.35%
Trade progress: 30000, 0.52%
Lobs progress: 10000, 0.39%
Trade progress: 40000, 0.70%
Trade progress: 50000, 0.87%
Trade progress: 60000, 1.05%
Trade progress: 70000, 1.22%
Lobs progress: 20000, 0.79%
Trade progress: 80000, 1.40%
Trade progress: 90000, 1.57%
Trade progress: 100000, 1.75%
Trade progress: 110000, 1.92%
Lobs progress: 30000, 1.18%
Trade progress: 120000, 2.10%
Trade progress: 130000, 2.27%
Lobs progress: 40000, 1.57%
Trade progress: 140000, 2.44%
Trade progress: 150000, 2.62%
Trade progress: 160000, 2.79%
Trade progress: 170000, 2.97%
Trade progress: 180000, 3.14%
Trade progress: 190000, 3.32%
Trade progress: 200000, 3.49%
Trade progress: 210000, 3.67%
Lobs progress: 50000, 1.97%
Trade progress: 220000, 3.84%
Trade progress: 230000, 4.02%
Trade progress: 240000, 4.19%
Lobs progress: 60000, 2.36%
Trade progress: 250000, 4.36%
Trade progress: 260000, 4.54%
Trade pr

Trade progress: 1770000, 30.90%
Trade progress: 1780000, 31.08%
Lobs progress: 920000, 36.20%
Trade progress: 1790000, 31.25%
Lobs progress: 930000, 36.59%
Trade progress: 1800000, 31.43%
Lobs progress: 940000, 36.99%
Trade progress: 1810000, 31.60%
Lobs progress: 950000, 37.38%
Trade progress: 1820000, 31.78%
Trade progress: 1830000, 31.95%
Trade progress: 1840000, 32.12%
Lobs progress: 960000, 37.78%
Trade progress: 1850000, 32.30%
Lobs progress: 970000, 38.17%
Trade progress: 1860000, 32.47%
Lobs progress: 980000, 38.56%
Trade progress: 1870000, 32.65%
Trade progress: 1880000, 32.82%
Lobs progress: 990000, 38.96%
Trade progress: 1890000, 33.00%
Trade progress: 1900000, 33.17%
Trade progress: 1910000, 33.35%
Trade progress: 1920000, 33.52%
Lobs progress: 1000000, 39.35%
Trade progress: 1930000, 33.70%
Trade progress: 1940000, 33.87%
Trade progress: 1950000, 34.04%
Trade progress: 1960000, 34.22%
Lobs progress: 1010000, 39.74%
Trade progress: 1970000, 34.39%
Trade progress: 1980000, 3

Trade progress: 3640000, 63.55%
Lobs progress: 1660000, 65.32%
Trade progress: 3650000, 63.73%
Trade progress: 3660000, 63.90%
Lobs progress: 1670000, 65.71%
Trade progress: 3670000, 64.07%
Trade progress: 3680000, 64.25%
Lobs progress: 1680000, 66.11%
Trade progress: 3690000, 64.42%
Trade progress: 3700000, 64.60%
Lobs progress: 1690000, 66.50%
Trade progress: 3710000, 64.77%
Trade progress: 3720000, 64.95%
Lobs progress: 1700000, 66.89%
Trade progress: 3730000, 65.12%
Trade progress: 3740000, 65.30%
Trade progress: 3750000, 65.47%
Lobs progress: 1710000, 67.29%
Trade progress: 3760000, 65.65%
Trade progress: 3770000, 65.82%
Lobs progress: 1720000, 67.68%
Trade progress: 3780000, 65.99%
Trade progress: 3790000, 66.17%
Lobs progress: 1730000, 68.07%
Trade progress: 3800000, 66.34%
Trade progress: 3810000, 66.52%
Trade progress: 3820000, 66.69%
Lobs progress: 1740000, 68.47%
Trade progress: 3830000, 66.87%
Trade progress: 3840000, 67.04%
Lobs progress: 1750000, 68.86%
Trade progress: 38

Trade progress: 5480000, 95.68%
Trade progress: 5490000, 95.85%
Trade progress: 5500000, 96.02%
Lobs progress: 2420000, 95.22%
Trade progress: 5510000, 96.20%
Trade progress: 5520000, 96.37%
Trade progress: 5530000, 96.55%
Lobs progress: 2430000, 95.62%
Trade progress: 5540000, 96.72%
Trade progress: 5550000, 96.90%
Lobs progress: 2440000, 96.01%
Trade progress: 5560000, 97.07%
Trade progress: 5570000, 97.25%
Lobs progress: 2450000, 96.41%
Trade progress: 5580000, 97.42%
Trade progress: 5590000, 97.60%
Lobs progress: 2460000, 96.80%
Trade progress: 5600000, 97.77%
Lobs progress: 2470000, 97.19%
Trade progress: 5610000, 97.94%
Trade progress: 5620000, 98.12%
Trade progress: 5630000, 98.29%
Lobs progress: 2480000, 97.59%
Trade progress: 5640000, 98.47%
Lobs progress: 2490000, 97.98%
Trade progress: 5650000, 98.64%
Trade progress: 5660000, 98.82%
Trade progress: 5670000, 98.99%
Lobs progress: 2500000, 98.37%
Trade progress: 5680000, 99.17%
Lobs progress: 2510000, 98.77%
Trade progress: 56

In [5]:
class Strategy:
    def __init__(self, max_position: float) -> None:
        pass

    def run(self, sim: "Sim"):
        while True:
            try:
                md_update = sim.tick()
                #call sim.place_order and sim.cancel_order here
            except StopIteration:
                break

In [ ]:
class Sim:
    def __init__(self, execution_latency: float, md_latency: float) -> None:
        self.md = iter(loaded_md)
        
        self.order_queue = []

    def tick(self) -> MdUpdate:
        self.execute_orders()
        self.prepare_orders()

        return next(self.md)

    def prepare_orders(self):
        pass

    def execute_orders(self):
        pass

    def place_order(self, side, size, price):
        return

    def cancel_order(self):
        pass

In [52]:
sim = Sim(10, 10)

for i in range(10):
    md = sim.tick()
    print(md)

KeyboardInterrupt: 